# 1. 匯入模組 - Import Modules & Rreload

In [1]:
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)

<module 'config.search_params' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\config\\search_params.py'>

# 2. 抓取Jobs清單 - Web Crawler

In [5]:
from crawler104 import Crawler104
import threaded_async_job
from config.search_params import get_filter_params
import numpy as np
import pandas as pd
import time

# keywords for filter again
keywords = r'工程|資料|python|data|數據|後端'

# custom filter params for search - sola
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}
# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}
filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)

# custom filter params for search - simon
# role = {'ro':'全職'}
# zone = {'zone': '4,5'}
# indcat = {'indcat':'1001000000,1002000000,1012000000,1014000000'}
# exclude = {'excludeJobKeyword': '軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶'}
# mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
# order = {'order':'日期排序'}
# asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, zone, indcat, exclude, mode, order, asc)

# 碼表 Start
start_time = time.time()
# 根據過濾參數開始爬蟲 & 再次根據keyword過濾jobs 
crawler104 = Crawler104(filter_params, keywords)
raw_jobs_104 = crawler104.search_job()
filter_jobs_104 = crawler104.filter_job(raw_jobs_104)
# 碼表 End
print(f"花費 {np.round((time.time() - start_time),2)} 秒")

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&area=6001002000%2C6001001000%2C6001005000%2C6001008000&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 79/79 [02:25<00:00,  1.84s/page]


載入79頁 | 載入2348筆資料 | 過濾後剩1826筆資料 | 花費 161.02 秒


# 3. 抓取Jobs內容(異步&多線程) - Web Scraper

In [356]:
filter_jobs = filter_jobs_104[200:280]

In [6]:
start_time = time.time()
# 工作網頁抓取 
jobs_details = threaded_async_job.scraper(filter_jobs_104)
print(f"花費 {np.round((time.time() - start_time),2)} 秒")

jobs:1826


Processing batches:   0%|          | 0/37 [00:00<?, ?batch/s]

Inventec_英業達股份有限公司:[Server]光學開發工程師(桃園):2024/01/18
國泰金控_國泰金融控股股份有限公司:雲端數據管理師- 雲端數據發展 (數數發中心, DDT)_I00015378:2024/01/18
PressPlay_瑞奧股份有限公司:PPA｜系統開發部｜資料工程師:2024/01/18
新安東京海上產物保險股份有限公司:數據管理人員:2024/01/19
保克曼有限公司:資深系統QA工程師:2024/01/18
英商米納創新有限公司在臺辦事處:後端工程師（可年後入職）:2024/01/17
英特拉金融科技股份有限公司:韌體工程師:2024/01/19
BookFast POS_新儀科技股份有限公司:APP 軟體開發工程師（iOS）:2024/01/16
麗清科技股份有限公司:軟體工程師(觀音):2024/01/18
奕垣科技有限公司:系統工程師:2024/01/19
DireSoft_德元軟體科技有限公司:遊戲後端工程師 / 資深:2024/01/17
Inventec_英業達股份有限公司:【研發替代役】AI 機器人工程師 (士林):2024/01/16
國泰金控_國泰金融控股股份有限公司:企業架構(EA)-軟體開發工程師 (前端工程師)_I00014719:2024/01/18
東森購物_電視購物_東森得易購股份有限公司:.NET工程師 / Backend Engineer (資訊研發部):2024/01/17
SGS _ 台灣檢驗科技股份有限公司:自動化測試開發工程師(台北):2024/01/16
鼎羿科技有限公司:Back-End Engineer 後端工程師:2024/01/16
敦謙國際智能科技股份有限公司:〔台中-西區〕Vue前端工程師:2024/01/19
知洋科技股份有限公司:韌體工程師(水下載具專案):2024/01/16
ViewSonic Corporation_優派國際股份有限公司:資深前端工程師,  Sr. Frontend Developer, PMSD:2024/01/17
哲煜科技股份有限公司:Junior  PHP Laravel  後端工程師┃台北:2024/01/17
華義國際數位娛樂股份有限公司:H5遊戲前端工程師:2024/01/18
山淬股份有限公司:電子商務 RPA 軟體工程師 (

Processing batches: 100%|██████████| 37/37 [03:31<00:00,  5.70s/batch]


花費 211.51 秒


In [91]:
data_list1 = [{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'name': '104人力銀行', 'item': 'https://www.104.com.tw/'}, {'@type': 'ListItem', 'position': 2, 'name': '雲育鏈股份有限公司', 'item': 'https://www.104.com.tw/company/1a2x6bld2j'}, {'@type': 'ListItem', 'position': 3, 'name': 'Python 開發工程師(台北)', 'item': 'https://www.104.com.tw/job/87e35'}]}, {'@context': 'http://schema.org', '@type': 'WebPage', 'name': 'Python 開發工程師(台北)', 'url': '//www.104.com.tw/job/87e35', 'relatedLink': 'https://www.104.com.tw/company/1a2x6bld2j', 'publisher': {'@type': 'Organization', 'name': '雲育鏈股份有限公司'}, 'significantLink': ['https://www.104.com.tw/company/1a2x6bld2j#info06']}, {'@context': 'http://schema.org', '@type': 'JobPosting', 'title': 'Python 開發工程師(台北)', 'url': '//www.104.com.tw/job/87e35?jobsource=google_job', 'datePosted': '2024/01/16', 'validThrough': '2025-01-17', 'description': '&lt;h3&gt;【工作內容】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;雲育鏈成立至今 4 年，致力於為台灣企業、金控提供有效雲端數位轉型方案，培訓了破千位的雲端、軟體人才，為台灣創造更多新型態企業與就業機會。&lt;br&gt;雲育鏈提供 AWS、GCP 雲端課程，並協助企業上雲。&lt;br&gt;合作的廠商包含金融產業、系統、遊戲、電信大廠等，提供內訓課程也提供一般標準化的課程。&lt;br&gt;&lt;br&gt;【工作內容】&lt;br&gt;設計和開發 Python 應用程式：Python 開發工程師的主要職責是設計和開發 Python 應用程式，包括 Web 應用程式、數據分析應用程式、機器學習應用程式等。&lt;br&gt;編寫高品質的程式碼：Python 開發工程師需要編寫高品質、易於維護的程式碼，並且了解程式碼編寫的最佳實踐和設計模式。&lt;br&gt;測試和調試程式碼：Python 開發工程師需要測試和調試程式碼，並且使用相應的測試框架和工具進行自動化測試。&lt;br&gt;整合第三方庫和 API：Python 開發工程師需要整合第三方庫和 API，以擴展 Python 的功能和特性，提高開發效率。&lt;br&gt;數據分析和機器學習：Python 是數據分析和機器學習領域的重要語言，Python 開發工程師需要了解這些領域的基礎知識和相關工具，並且能夠應用這些知識開發相應的應用程式。&lt;br&gt;敏捷開發方法論：Python 開發工程師需要了解敏捷開發方法論和相應的工具，如 Scrum、Kanban 等，以實現項目快速迭代和交付。&lt;br&gt;團隊協作能力：Python 開發工程師需要具備良好的團隊協作能力，能夠與其他開發人員、項目經理和客戶進行有效的溝通和協調，共同完成項目的開發和交付。&lt;br&gt;&lt;br&gt;- 職務類別：軟體工程師,其他資訊專業人員,軟體專案主管&lt;br&gt;- 工作待遇：待遇面議&lt;br&gt;- 工作性質：全職&lt;br&gt;- 上班地點：台北市中山區&lt;br&gt;- 管理責任：不需負擔管理責任&lt;br&gt;- 出差外派：需出差，一年累積時間約一個月以下&lt;br&gt;- 上班時段：日班/晚班&lt;br&gt;- 休假制度：週休二日&lt;br&gt;- 可上班日：不限&lt;br&gt;- 需求人數：1~2人&lt;br&gt;&lt;br&gt;&lt;h3&gt;【相關條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 工作經歷：2年以上&lt;br&gt;- 學歷要求：專科、大學、碩士&lt;br&gt;- 公司產業：其它軟體及網路相關業&lt;br&gt;- 科系要求：不拘&lt;br&gt;- 語文條件：不拘&lt;br&gt;- 擅長工具：Linux,Java,Python,AWS&lt;br&gt;- 工作技能：軟體工程系統開發,軟體程式設計,網路程式設計,資料庫系統管理維護,資料庫程式設計&lt;br&gt;&lt;br&gt;&lt;h3&gt;【其他條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 其他：1. Python 編程能力：Python 工程師需要熟悉 Python 語言的基礎語法、標準庫和常用的第三方庫，能夠使用 Python 編程開發各種類型的應用程式。&lt;br&gt;2. Web 開發：Python 是 Web 開發的重要語言之一，Python 工程師需要了解 Web 開發相關的知識，包括 Web 框架、HTTP 協議、前端開發等。&lt;br&gt;3. 數據分析和機器學習：Python 是數據分析和機器學習領域的重要語言之一，Python 工程師需要了解數據分析和機器學習的基礎知識和相關庫，如 NumPy、Pandas、Scikit-learn 等。&lt;br&gt;4. 敏捷開發方法論：Python 工程師需要了解敏捷開發方法論和相應的工具，如 Scrum、Kanban 等，以實現項目快速迭代和交付。&lt;br&gt;5. 資料庫：Python 工程師需要了解資料庫相關的知識，如 SQL 語言、關係型資料庫、NoSQL 資料庫等。&lt;br&gt;6. Linux/Unix 系統管理：Python 工程師需要了解 Linux/Unix 系統的基本操作和管理，能夠在 Linux/Unix 系統上部署和運行 Python 應用程式。&lt;br&gt;7. 基本的數學和統計學知識：Python 工程師需要了解基本的數學和統計學知識，以應用於數據分析和機器學習等領域。&lt;br&gt;&lt;br&gt;&lt;h3&gt;【公司福利】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 福利制度：1. 基本福利：勞健保、勞退&lt;br&gt;2. 經常性 Foodpanda 點心、飲料、宵夜&lt;br&gt;3. 公司零食&lt;br&gt;4. 歡樂輕鬆的辦公室氣氛&lt;br&gt;5. 經常性公司聚餐餐會&lt;br&gt;6. 季目標獎金 / 行銷 / 業務績效獎金&lt;br&gt;7. 不定期員工旅遊&lt;br&gt;8. 年終獎金&lt;br&gt;9. 有機會認識業界行銷、雲端、資訊、AWS原廠人脈&lt;br&gt;10. 有機會在業界曝光自己個人形象&lt;br&gt;11. 全體員工皆為「下午晚上」上班，可充分利用上午黃金時間&lt;br&gt;12. 公司尊重員工職涯發展意願&lt;br&gt;13. 員工與老闆皆對外誠實做自己&lt;br&gt;14. 特休&lt;br&gt;年終獎金,零食櫃,國內旅遊,部門聚餐,免費下午茶,下午茶&lt;br&gt;&lt;br&gt;&lt;br&gt;更多工作資訊請參考：https://www.104.com.tw/job/87e35', 'employmentType': 'FULL_TIME', 'hiringOrganization': {'@type': 'Organization', 'name': '雲育鏈股份有限公司', 'sameAs': 'https://www.104.com.tw/company/1a2x6bld2j', 'logo': 'https://static.104.com.tw/b_profile/cust_picture/7803/130000000157803/logo.png?v=20201011162918'}, 'identifier': {'@type': 'PropertyValue', 'name': '104人力銀行_一零四資訊科技股份有限公司', 'value': '87e35'}, 'jobLocation': {'@type': 'Place', 'address': {'@type': 'PostalAddress', 'streetAddress': '長安東路2段80號5F', 'addressLocality': '台北市中山區', 'postalCode': '104', 'addressRegion': '台灣', 'addressCountry': 'TW'}}, 'baseSalary': {'@type': 'MonetaryAmount', 'currency': 'TWD', 'value': {'@type': 'QuantitativeValue', 'unitText': 'MONTH', 'value': '40000元以上'}}, 'mainEntityOfPage': {'@type': 'WebPage', '@id': '//www.104.com.tw/job/87e35'}, 'educationRequirements': {'@type': 'EducationalOccupationalCredential', 'credentialCategory': 'professional certificate, bachelor degree, postgraduate degree'}, 'experienceRequirements': {'@type': 'OccupationalExperienceRequirements', 'monthsOfExperience': 24}, 'industry': '其它軟體及網路相關業', 'workHours': '日班/晚班', 'skills': ['軟體工程系統開發', '軟體程式設計', '網路程式設計', '資料庫系統管理維護', '資料庫程式設計']}]

In [94]:
data_list2 = [{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'name': '104人力銀行', 'item': 'https://www.104.com.tw/'}, {'@type': 'ListItem', 'position': 2, 'name': '奇策智能雲端股份有限公司', 'item': 'https://www.104.com.tw/company/1a2x6bliq7'}, {'@type': 'ListItem', 'position': 3, 'name': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'item': 'https://www.104.com.tw/job/82ua3'}]}, {'@context': 'http://schema.org', '@type': 'WebPage', 'name': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'url': '//www.104.com.tw/job/82ua3', 'relatedLink': 'https://www.104.com.tw/company/1a2x6bliq7', 'publisher': {'@type': 'Organization', 'name': '奇策智能雲端股份有限公司'}, 'significantLink': ['https://www.104.com.tw/company/1a2x6bliq7#info06']}, {'@context': 'http://schema.org', '@type': 'JobPosting', 'title': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'url': '//www.104.com.tw/job/82ua3?jobsource=google_job', 'datePosted': '2024/01/16', 'validThrough': '2025-01-17', 'description': '&lt;h3&gt;【工作內容】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 徵求在學學生或應屆畢業生&lt;br&gt;- 區塊鏈相關 WEB APP 開發與維護。&lt;br&gt;- ITRI、Apple、Tesla 技術高手親自指導，幫助你學習國外最新 SaaS 開發的觀念與思維。&lt;br&gt;- 學習量化交易知識與免費使用公司自動交易服務。 &lt;br&gt;- 表現優異者，有機會轉 Paid Intern 或正職！ &lt;br&gt;&lt;br&gt;※ 請注意：此為學習型無薪實習，為期三個月，可配合學校簽訂契約、實習證明和推薦信（表現需具備一定水準者），無法接受者，請勿投遞，謝謝！&lt;br&gt;※ 需通過遠端線下 coding 測驗（CSS + React 各一題）&lt;br&gt;&lt;br&gt;&lt;br&gt;- 職務類別：Internet程式設計師,軟體工程師&lt;br&gt;- 工作待遇：論件計酬0元&lt;br&gt;- 工作性質：全職&lt;br&gt;- 上班地點：台北市中正區&lt;br&gt;- 管理責任：不需負擔管理責任&lt;br&gt;- 出差外派：無需出差外派&lt;br&gt;- 上班時段：日班/晚班/假日班，彈性配合&lt;br&gt;- 休假制度：週休二日&lt;br&gt;- 可上班日：不限&lt;br&gt;- 需求人數：不限&lt;br&gt;&lt;br&gt;&lt;h3&gt;【相關條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 工作經歷：不拘&lt;br&gt;- 學歷要求：高中以上&lt;br&gt;- 公司產業：電腦系統整合服務業&lt;br&gt;- 科系要求：資訊工程相關,其他數學及電算機科學相關&lt;br&gt;- 語文條件：英文--聽 /略懂、說 /略懂、讀 /精通、寫 /中等&lt;br&gt;- 擅長工具：Github,JavaScript,CSS,ReactJS&lt;br&gt;- 工作技能：軟體程式設計,網路程式設計&lt;br&gt;&lt;br&gt;&lt;h3&gt;【其他條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 其他：對以下領域有興趣者，歡迎應徵：&lt;br&gt;- Advanced Web Front-end Development&lt;br&gt;- Blockchain and Smart Contract&lt;br&gt;- Microservice Technology&lt;br&gt;- Machine Learning&lt;br&gt;- Cybersecurity&lt;br&gt;- 加密貨幣投資&lt;br&gt;&lt;br&gt;實習期間，你可以學習到：&lt;br&gt;- 技術分析&lt;br&gt;- 量化交易自動化&lt;br&gt;- 程式交易策略開發&lt;br&gt;- Agile / Scrum 開發實戰&lt;br&gt;- 前端原子設計框架&lt;br&gt;- React.js (Next.js)&lt;br&gt;- HTML / CSS (SCSS)&lt;br&gt;- TypeScript&lt;br&gt;- Node.js (ES6)&lt;br&gt;- Apollo GraphQL Client (react-graphql)&lt;br&gt;- ...&lt;br&gt;&lt;br&gt;&lt;br&gt;&lt;h3&gt;【公司福利】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 福利制度：◎薪酬&lt;br&gt;•\t具市場競爭性的薪資&lt;br&gt;•\t員工免費股票&lt;br&gt;•\t員工低價認股權&lt;br&gt;•\t員工私募輪認購 Crypto-Arsenal Token 計畫&lt;br&gt;&lt;br&gt;◎工作、學習與生活並重&lt;br&gt;•\t遠距工作為主、定期面對面工作計畫與檢討為輔&lt;br&gt;•\t定期部門聚餐&lt;br&gt;•\t線上課程補助&lt;br&gt;&lt;br&gt;◎保險&lt;br&gt;•\t享勞健保&lt;br&gt;&lt;br&gt;◎休假制度&lt;br&gt;•\t每年 14 天給薪假&lt;br&gt;員工認股,入股分紅,不扣薪病假,部門聚餐,不用補班&lt;br&gt;&lt;br&gt;&lt;br&gt;更多工作資訊請參考：https://www.104.com.tw/job/82ua3', 'employmentType': 'FULL_TIME', 'hiringOrganization': {'@type': 'Organization', 'name': '奇策智能雲端股份有限公司', 'sameAs': 'https://www.104.com.tw/company/1a2x6bliq7', 'logo': 'https://static.104.com.tw/b_profile/cust_picture/5135/130000000165135/logo.png?v=20210306085049'}, 'identifier': {'@type': 'PropertyValue', 'name': '104人力銀行_一零四資訊科技股份有限公司', 'value': '82ua3'}, 'jobLocation': {'@type': 'Place', 'address': {'@type': 'PostalAddress', 'streetAddress': '遠端工作', 'addressLocality': '台北市中正區', 'postalCode': '100', 'addressRegion': '台灣', 'addressCountry': 'TW'}}, 'baseSalary': {'@type': 'MonetaryAmount', 'currency': 'TWD', 'value': {'@type': 'QuantitativeValue', 'unitText': 'DAY', 'value': '論件計酬0元'}}, 'mainEntityOfPage': {'@type': 'WebPage', '@id': '//www.104.com.tw/job/82ua3'}, 'educationRequirements': {'@type': 'EducationalOccupationalCredential', 'credentialCategory': 'high school'}, 'industry': '電腦系統整合服務業', 'workHours': '日班/晚班/假日班，彈性配合', 'skills': ['軟體程式設計', '網路程式設計']}]

In [553]:
# 測試
# data_list1

# 99. 測試區

In [50]:
len(jobs_details)
from datetime import datetime
import os
import xlsxwriter
current_date = datetime.now().date()    
user = "sola"
counter = 1

output_filename = f'output/{user}_{current_date}.xlsx'
base_filename = f"{user}_{current_date}"

jobs_details.replace(r'=\w+', '', regex=True, inplace=True)


while os.path.exists(output_filename):
    output_filename = f'output/{user}_{current_date}_{counter}.xlsx'
    base_filename = f"{user}_{current_date}_{counter}"
    counter += 1

print(output_filename)
# 去除文件扩展名
try:

    jobs_details.to_excel(output_filename, sheet_name=base_filename, index=False, engine='xlsxwriter')
    print(f"CSV文件保存成功: {output_filename}")
except PermissionError as e:
    print(f"无法保存文件: {e}")




output/sola_2024-01-19_9.xlsx
CSV文件保存成功: output/sola_2024-01-19_9.xlsx


In [556]:
# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，
# gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。

import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)

Start asynchronous task 0
Start asynchronous task 1
Start asynchronous task 2
Start asynchronous task 3
Start asynchronous task 4
End asynchronous task 0
End asynchronous task 2
End asynchronous task 4
End asynchronous task 1
End asynchronous task 3
['Result of task 0', 'Result of task 1', 'Result of task 2', 'Result of task 3', 'Result of task 4']
